In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/miramedix')

In [ ]:
pip install ml_metrics

In [ ]:
from py_metrics import topNacc, acc_scores, get_rr_score, mean_rr, ndcg_score
import ml_metrics
import numpy as np
import pandas as pd
import json
import sklearn
import math

# annotations preprocess (LB data), getting true_recs



In [ ]:
import json

with open('annotation_3.json', 'r') as f: #annotation_N.json where N = 1,2,3
  data_annot = json.load(f)

In [ ]:
results = {}

for i in data_annot:
  idx = i['id']
  choices = i['annotations'][0]['result'][0]['value']['choices']
  results[idx] = choices

In [ ]:
len(results.keys())

74

In [ ]:
#for i in range(9187, 9262): #for annot 1
#for i in range(9262, 9337): #for annot 2
for i in range(9457, 9532):
  if i in results.keys():
    continue
  else:
    results[i] = []

In [ ]:
with open('annot_results_3.json', 'w') as f: #saved annot_results_N.json where N = 1,2,3
  json.dump(results, f)

Got true ids, now get papers

umls 1 ver

In [ ]:
import json

with open('annot_results_1.json', 'r') as f:
  results = json.load(f)

In [ ]:
with open('texts_recs_with_urls.json', 'r') as f:
  data = json.load(f)

In [ ]:
data['1']

['<h1><span class="qt0"><span class="qt1">High Blood Pressure</span></span></h1> \nhttps://medlineplus.gov/highbloodpressure.html\nWhat is blood pressure?<p>Blood pressure is the force of your blood pushing against the walls of your arteries. Each time your heart beats, it pumps blood into the arteries. Your <span class="qt0"><span class="qt1">blood pressure is highest</span></span> when your heart beats, pumping the blood. This is called systolic pressure. When your heart is at rest, between beats, your blood pressure falls. This is called diastolic pressure.</p><p>Your blood pressure reading uses these two numbers. Usually the systolic number comes before or above the diastolic number. For example, 120/80 means a systolic of 120 and a diastolic of 80.</p>How is <span class="qt0"><span class="qt1">high blood pressure</span></span> diagnosed?<p><span class="qt0"><span class="qt1">High blood pressure</span></span> usually has no symptoms. So the only way to find out if you have it is to

In [ ]:
res_sorted = sorted(results.items(), key=lambda x: x[0])
res_sorted[0]

(9187, ['1', '3', '5', '6'])

In [ ]:
len(res_sorted)

75

In [ ]:
true_recs = {}

for i in range(len(res_sorted)):
  text_recs = []
  
  for j in res_sorted[i][1]:
    idx = int(j) - 1
    text_recs.append(data[str(i+1)][idx])  
  true_recs[str(i+1)] = text_recs

In [ ]:
true_recs['1']

['<h1><span class="qt0"><span class="qt1">High Blood Pressure</span></span></h1> \nhttps://medlineplus.gov/highbloodpressure.html\nWhat is blood pressure?<p>Blood pressure is the force of your blood pushing against the walls of your arteries. Each time your heart beats, it pumps blood into the arteries. Your <span class="qt0"><span class="qt1">blood pressure is highest</span></span> when your heart beats, pumping the blood. This is called systolic pressure. When your heart is at rest, between beats, your blood pressure falls. This is called diastolic pressure.</p><p>Your blood pressure reading uses these two numbers. Usually the systolic number comes before or above the diastolic number. For example, 120/80 means a systolic of 120 and a diastolic of 80.</p>How is <span class="qt0"><span class="qt1">high blood pressure</span></span> diagnosed?<p><span class="qt0"><span class="qt1">High blood pressure</span></span> usually has no symptoms. So the only way to find out if you have it is to

In [ ]:
true_recs_noemp = {}
for i in true_recs.keys():
  if len(true_recs[i]) != 0:
    true_recs_noemp[i] = true_recs[i]
  else:
    print(i)

22
66
67


In [ ]:
len(true_recs_noemp)

72

In [ ]:
with open('true_recs.json', 'w') as f:
  json.dump(true_recs, f)

In [ ]:
with open('true_recs_noemp.json', 'w') as f:
  json.dump(true_recs_noemp, f)

umls 2 ver (reranked data)

немного кода с сохранением переранжированных статей после pyserini

In [ ]:
with open('new_recs_raw.json', 'r') as f:
  new_recs = json.load(f)

In [ ]:
rec_nums = {}
for i in new_recs:
  recs_clean = []
  for rec in new_recs[i]:
    recs_clean.append(rec[0][3:])
  rec_nums[i] = recs_clean

rec_nums['1']

['11', '2', '1', '3', '4', '5', '9', '7', '6', '10']

In [ ]:
reranked_recs = {}
for i in data.keys():
  new_list = []
  for paper_id in rec_nums[i]:
    idx = int(paper_id) - 1
    new_list.append(data[i][idx])
  reranked_recs[i] = new_list

In [ ]:
reranked_recs['1']

['<h1>Sciatica</h1> \nhttps://medlineplus.gov/sciatica.html\n<p>Sciatica is a symptom of a problem with the sciatic nerve, the largest nerve in the body. It controls muscles in the back of your knee and lower leg and provides feeling to the back of your <span class="qt0">thigh</span>, part of your lower leg, and the sole of your foot. When you have sciatica, you have pain, weakness, <span class="qt1">numbness</span>, or tingling. It can start in the lower back and extend down your leg to your calf, foot, or even your toes. It\'s usually on only one side of your body.</p><p>Causes of sciatica include:</p><ul><li>A ruptured intervertebral disk</li><li>Narrowing of the spinal canal that puts pressure on the nerve, called spinal stenosis</li><li>An injury such as a pelvic fracture.</li></ul><p>In many cases no cause can be found.</p><p>Sometimes sciatica goes away on its own. Treatment, if needed, depends on the cause of the problem. It may include exercises, medicines, and surgery.</p>\n'

In [ ]:
with open('text_recs_reranked.json', 'w') as f:
  json.dump(reranked_recs, f)

Извлекаем true статьи

In [ ]:
with open('text_recs_reranked.json', 'r') as f:
  reranked_recs = json.load(f)

In [ ]:
with open('annot_results_2.json', 'r') as f:
  results = json.load(f)

In [ ]:
res_sorted = sorted(results.items(), key=lambda x: x[0])
res_sorted[0]

('9262', ['3', '4', '6', '9'])

In [ ]:
len(res_sorted)

75

In [ ]:
true_recs = {}

for i in range(len(res_sorted)):
  text_recs = []
  for j in res_sorted[i][1]:
    idx = int(j) - 1
    text_recs.append(reranked_recs[str(i+1)][idx])
  
  true_recs[str(i+1)] = text_recs

In [ ]:
true_recs['1']

['<h1><span class="qt0"><span class="qt1">High Blood Pressure</span></span></h1> \nhttps://medlineplus.gov/highbloodpressure.html\nWhat is blood pressure?<p>Blood pressure is the force of your blood pushing against the walls of your arteries. Each time your heart beats, it pumps blood into the arteries. Your <span class="qt0"><span class="qt1">blood pressure is highest</span></span> when your heart beats, pumping the blood. This is called systolic pressure. When your heart is at rest, between beats, your blood pressure falls. This is called diastolic pressure.</p><p>Your blood pressure reading uses these two numbers. Usually the systolic number comes before or above the diastolic number. For example, 120/80 means a systolic of 120 and a diastolic of 80.</p>How is <span class="qt0"><span class="qt1">high blood pressure</span></span> diagnosed?<p><span class="qt0"><span class="qt1">High blood pressure</span></span> usually has no symptoms. So the only way to find out if you have it is to

In [ ]:
with open('true_recs_2.json', 'w') as f:
  json.dump(true_recs, f)

Join all true recs (it was used for research purposes)

In [ ]:
with open('true_recs.json', 'r') as f:
  true_1 = json.load(f)

with open('true_recs_2.json', 'r') as f:
  true_2 = json.load(f)

In [ ]:
all_true = {}
true_1_clean = {}
true_2_clean = {}

for key in true_1.keys():
  all_true[key] = true_1[key]
  all_true[key].extend(true_2[key])
  all_true[key] = list(set(all_true[key]))

  true_1_clean[key] = list(set(true_1[key]))
  true_2_clean[key] = list(set(true_2[key]))

In [ ]:
for i in all_true.keys():
  if len(all_true[i]) == 0:
    print(i)

67


In [ ]:
with open('true_recs_clean.json', 'w') as f:
  json.dump(true_1_clean, f)

with open('true_recs_2_clean.json', 'w') as f:
  json.dump(true_2_clean, f)

with open('true_recs_all.json', 'w') as f:
  json.dump(all_true, f)

3 ver: bert + umls

In [ ]:
with open('texts_recs_umls_m3_2_clean_reranked.json', 'r') as f:
  data = json.load(f)

In [ ]:
with open('annot_results_3.json', 'r') as f:
  results = json.load(f)

In [ ]:
res_sorted = sorted(results.items(), key=lambda x: x[0])
res_sorted[0]

('9457', ['3', '6', '8', '9'])

In [ ]:
true_recs = {}

for i in range(len(res_sorted)):
  text_recs = []
  for j in res_sorted[i][1]:
    idx = int(j) - 1
    try:
      text_recs.append(data[str(i+1)][idx])
    except:      
      continue
  
  true_recs[str(i+1)] = text_recs

In [ ]:
true_recs['1']

['<h1><span class="qt0">Chronic</span> <span class="qt1">Pain</span></h1> \nhttps://medlineplus.gov/chronicpain.html\n<p><span class="qt1">Pain</span> is a signal in your nervous system that something may be wrong.  It is an unpleasant feeling, such as a prick, tingle, sting, burn, or <span class="qt1">ache. Pain</span> may be sharp or dull. You may feel <span class="qt1">pain</span> in one area of your body, or all over.  There are two types: acute <span class="qt1">pain</span> and <span class="qt0">chronic</span> <span class="qt1">pain</span>. Acute <span class="qt1">pain</span> lets you know that you may be injured or a have problem you need to take care of. <span class="qt0">Chronic</span> <span class="qt1">pain</span> is different.  The <span class="qt1">pain</span> may last for weeks, months, or even years. The original cause may have been an injury or infection. There may be an ongoing cause of <span class="qt1">pain</span>, such as arthritis or cancer. In some cases there is no

In [ ]:
with open('true_recs_3.json', 'w') as f:
  json.dump(true_recs, f)

In [ ]:
true_3_clean = {}

for key in true_recs.keys():

  true_3_clean[key] = list(set(true_recs[key]))
  if len(true_recs[key]) != len(true_3_clean[key]):
    print(len(true_recs[key]), len(true_3_clean[key]))

проверка на повторы, повторов нет

# model recs preprocess (no repetitions)

In [ ]:
import os
models = os.listdir("models_recs_2ver/")
models[:5]

['texts_recs_m5_2_per3.json',
 'texts_recs_m5_2_filtered_per3.json',
 'texts_recs_m3_per3.json',
 'texts_recs_m3_filtered_per3.json',
 'texts_recs_m4_per3.json']

In [ ]:
def clean_results(model):
  new_results = {}

  with open(f'models_recs_2ver/{model}', 'r') as f:
    model_preds = json.load(f)
  
  for key in model_preds.keys():
    new_list = []
    for paper in model_preds[key]:
      if paper not in new_list:
        new_list.append(paper)
    new_results[key] = new_list

  return new_results 

In [ ]:
for model in models:
  new_model_data = clean_results(model)

  with open(f'models_recs_2ver_clean/{model}', 'w') as f:
    json.dump(new_model_data, f)

# models' ner additional preprocess

In [ ]:
import os
ner_res = os.listdir("ner_results_2ver/filtered")
ner_res[:5]

['ner_m5_2.json',
 'ner_m5_2_filtered.json',
 'ner_m3.json',
 'ner_m3_filtered.json',
 'ner_m4.json']

In [ ]:
'the tumor'[4:]

'tumor'

In [ ]:
for ner in ner_res:

  clean_all_ner = {}
  with open(f'ner_results_2ver/filtered/{ner}', 'r') as f:
    all_ner = json.load(f)
  
  for key in all_ner.keys():
    key_ner = []
    for entity in all_ner[key]:
      entity = re.sub(' / ', '/', entity.lower())
      if entity.startswith('a '):
        key_ner.append(entity[2:])
      elif entity.startswith('the '):
        key_ner.append(entity[4:])
      else:
        key_ner.append(entity)
    clean_all_ner[key] = key_ner
  
  with open(f'ner_results_2ver/filtered_clean/{ner}', 'w') as f:
    json.dump(clean_all_ner, f)

In [ ]:
len(ner_res), len(os.listdir("ner_results_2ver/filtered_clean"))

(30, 30)

# umls and true ner preprocess

In [ ]:
with open('terms_filtered_4.json', 'r') as f:
  annot_data = json.load(f)

In [ ]:
for i in annot_data['1'][0]:
  print(i['ngram'])

anaplastic astrocytoma
anaplastic astrocytoma
hypertension
tumor
thigh numbness


In [ ]:
umls_terms = {}
for i in annot_data.keys():
  umls_terms[i] = list(set([re.sub(' / ', '/', j['ngram'].lower()) for j in annot_data[i][0]]))

In [ ]:
for key in umls_terms.keys():
  if len(umls_terms[key]) != len(list(set(umls_terms[key]))):
    print('repeat')

In [ ]:
with open(f'umls_ngrams.json', 'w') as f:
    json.dump(umls_terms, f)

True ner preprocess (done for all 4 versions)

In [ ]:
with open('true_ner_results_4.json', 'r') as f:
  true_terms = json.load(f)

In [ ]:
import re

true_terms_clean = {}
for key in true_terms.keys():
  terms_clean = [re.sub(' / ', '/', term.lower()) for term in true_terms[key]]
  true_terms_clean[key] = terms_clean

In [ ]:
with open(f'true_ner_results_4_clean.json', 'w') as f:
    json.dump(true_terms_clean, f)

# ner evaluation

In [ ]:
with open('true_ner_results_clean.json', 'r') as f:
  true_terms = json.load(f)

In [ ]:
true_terms_list = [true_terms[str(i)] for i in range(1,76)]
true_terms_list[0]

['tumor',
 'unresectable',
 'anaplastic astrocytoma',
 'radiation',
 't-l spine',
 'temozolomide',
 'field radiation',
 'biopsied',
 'spinal cord conus mass',
 'rle weakness',
 'right knee',
 'difficulty walking',
 'right anterior thigh numbness',
 'lower extremity weakness',
 'urinary retention',
 'steroids',
 'anaplastic astrocytoma of the spine',
 'hypertension',
 'chronic pain',
 'foley catheter']

In [ ]:
with open('umls_ngrams.json', 'r') as f:
  umls_terms = json.load(f)

In [ ]:
umls_terms_list = [umls_terms[str(i)] for i in range(1,76)]
umls_terms_list[0]

['anaplastic astrocytoma', 'tumor', 'hypertension', 'thigh numbness']

In [ ]:
import os
ner_res = os.listdir("ner_results_2ver/filtered_clean")

In [ ]:
def evaluate(true_kws, predicted_kws):
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        if union == 0:
          jac == 0
        else:
          jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    
    mean_precision = round(np.mean(precisions), 2) if math.isnan(round(np.mean(precisions),2)) == False else float(0)
    mean_recall = round(np.mean(recalls), 2) if math.isnan(round(np.mean(recalls), 2)) == False else float(0)
    mean_f1 = round(np.mean(f1s), 2) if math.isnan(round(np.mean(f1s),2)) == False else float(0)
    mean_jaccard = round(np.mean(jaccards), 2) if math.isnan(round(np.mean(jaccards),2)) == False else float(0)

    return mean_precision, mean_recall, mean_f1, mean_jaccard

In [ ]:
evaluate(true_terms_list, true_terms_list)

(1.0, 1.0, 1.0, 1.0)

Оценка UMLS

In [ ]:
evaluate(true_terms_list, umls_terms_list)

(0.55, 0.23, 0.31, 0.19)

Оценка результатов моделей

In [ ]:
def models_scores(models, true_data):

  model_names = []
  p_scores = []
  r_scores = []
  f_scores = []
  j_scores = []

  for model in models:
  
    model_names.append(model[4:-5])

    with open(f'ner_results_2ver/filtered_clean/{model}', 'r') as f:
      model_data = json.load(f)
    
    data_list = [model_data[str(i)] for i in range(1,76)]
    #mean_len = np.mean([len(all_ner[str(i)]) for i in range(1,76)])
    #print(mean_len)

    mean_precision, mean_recall, mean_f1, mean_jaccard = evaluate(true_data, data_list)

    p_scores.append(mean_precision)
    r_scores.append(mean_recall)
    f_scores.append(mean_f1)
    j_scores.append(mean_jaccard)
  
  return model_names, p_scores, r_scores, f_scores, j_scores

In [ ]:
model_names, p_scores, r_scores, f_scores, j_scores = models_scores(ner_res, true_terms_list)

In [ ]:
import pandas as pd
import math

metric_names = ['mean_precision', 'mean_recall', 'mean_f1', 'mean_jaccard']

ner_scores_df = pd.DataFrame()
ner_scores_df['models'] = model_names
ner_scores_df['precision'] = p_scores
ner_scores_df['recall'] = r_scores
ner_scores_df['f1_score'] = f_scores
ner_scores_df['jaccard'] = j_scores

In [ ]:
ner_scores_df.head()

,models,precision,recall,f1_score,jaccard
0,m5_2,0.11,0.34,0.17,0.09
1,m5_2_filtered,0.11,0.34,0.17,0.09
2,m3,0.28,0.06,0.09,0.05
3,m3_filtered,0.28,0.06,0.09,0.05
4,m4,0.45,0.05,0.09,0.05


In [ ]:
max(ner_scores_df['precision']), max(ner_scores_df['recall']), max(ner_scores_df['f1_score']), max(ner_scores_df['jaccard'])

(0.55, 0.46, 0.41, 0.27)

In [ ]:
ner_scores_df['precision'].idxmax(), ner_scores_df['recall'].idxmax(), ner_scores_df['f1_score'].idxmax(), ner_scores_df['jaccard'].idxmax()

(18, 16, 16, 16)

In [ ]:
ner_scores_df.loc[[18]]

,models,precision,recall,f1_score,jaccard
18,m4_2,0.55,0.19,0.27,0.17


In [ ]:
ner_scores_df.loc[[16]]

,models,precision,recall,f1_score,jaccard
16,m3_2,0.4,0.46,0.41,0.27


В среднем у модели m3_2 получились лучшие результаты (высокие и сбалансированные)

Попробуем объединить umls и m3_2

In [ ]:
with open(f'ner_results_2ver/filtered_clean/ner_m3_2.json', 'r') as f:
      m3_data = json.load(f)

In [ ]:
mixed_terms = {}
for i in umls_terms.keys():
  mixed_terms[i] = umls_terms[i]
  for term in m3_data[i]:
    if term not in umls_terms[i]:
      mixed_terms[i].append(term)

In [ ]:
with open(f'umls_m3_2_terms.json', 'w') as f:
    json.dump(mixed_terms, f)

In [ ]:
test_list = [mixed_terms[str(i)] for i in range(1,76)]

UMLS + m3_2

In [ ]:
evaluate(test_list, true_terms_list)

(0.55, 0.4, 0.44, 0.3)

UMLS

In [ ]:
evaluate(true_terms_list, umls_terms_list)

(0.55, 0.23, 0.31, 0.19)

m3_2

In [ ]:
ner_scores_df.loc[[16]]

,models,precision,recall,f1_score,jaccard
16,m3_2,0.4,0.46,0.41,0.27


Проверим разные варианты true terms

In [ ]:
with open('true_ner_results_2_clean.json', 'r') as f:
  true_terms = json.load(f)

true_terms_list = [true_terms[str(i)] for i in range(1,76)]
true_terms_list[0]

evaluate(test_list, true_terms_list)

(0.55, 0.4, 0.44, 0.3)

In [ ]:
with open('true_ner_results_3_clean.json', 'r') as f:
  true_terms = json.load(f)

true_terms_list = [true_terms[str(i)] for i in range(1,76)]
true_terms_list[0]

evaluate(test_list, true_terms_list)

(0.55, 0.4, 0.45, 0.3)

In [ ]:
with open('true_ner_results_4_clean.json', 'r') as f:
  true_terms = json.load(f)

true_terms_list = [true_terms[str(i)] for i in range(1,76)]
true_terms_list[0]

evaluate(test_list, true_terms_list)

(0.46, 0.45, 0.44, 0.29)

Наилучший результат у третьего варианта (true_ner_results_3_clean.json)

# recommentations rerank

Здесь выполняется переранжирование с использованием TfidfVectorizer (переранжирование с Pyserini осуществлялось в тетрадке pyserini_rerank.ipynb)

Дополнительная обработка для третьей системы (QuickUMLS + BERT)

In [ ]:
def reps_prepro(model):
  new_results = {}

  with open(f'{model}', 'r') as f:
    model_preds = json.load(f)
  
  for key in model_preds.keys():
    new_list = []
    links = []
    for paper in model_preds[key]:
      paper_data = paper.split('\n')
      link = paper_data[1]
      if link not in links:
        new_list.append(paper)
        links.append(link)
    new_results[key] = new_list

  return new_results

In [ ]:
new_res_1 = reps_prepro('texts_recs_umls_m3_2.json')

with open('texts_recs_umls_m3_2_clean.json', 'w') as f:
  json.dump(new_res_1, f)

In [ ]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
with open('descriptions.json', 'r') as f:
  queries = json.load(f)

In [ ]:
queries['1']

'\nPatient is a 45-year-old man with a history of anaplastic astrocytoma of the spine complicated by severe lower extremity weakness and urinary retention s/p Foley catheter, high-dose steroids, hypertension, and chronic pain. The tumor is located in the T-L spine, unresectable anaplastic astrocytoma s/p radiation. Complicated by progressive lower extremity weakness and urinary retention. Patient initially presented with RLE weakness where his right knee gave out with difficulty walking and right anterior thigh numbness. MRI showed a spinal cord conus mass which was biopsied and found to be anaplastic astrocytoma. Therapy included field radiation t10-l1 followed by 11 cycles of temozolomide 7 days on and 7 days off. This was followed by CPT-11 Weekly x4 with Avastin Q2 weeks/ 2 weeks rest and repeat cycle. \n'

In [ ]:
queries_clean = {}
for key in queries.keys():
  clean_text = queries[key].strip()
  queries_clean[key] = clean_text

In [ ]:
queries_clean['1']

'Patient is a 45-year-old man with a history of anaplastic astrocytoma of the spine complicated by severe lower extremity weakness and urinary retention s/p Foley catheter, high-dose steroids, hypertension, and chronic pain. The tumor is located in the T-L spine, unresectable anaplastic astrocytoma s/p radiation. Complicated by progressive lower extremity weakness and urinary retention. Patient initially presented with RLE weakness where his right knee gave out with difficulty walking and right anterior thigh numbness. MRI showed a spinal cord conus mass which was biopsied and found to be anaplastic astrocytoma. Therapy included field radiation t10-l1 followed by 11 cycles of temozolomide 7 days on and 7 days off. This was followed by CPT-11 Weekly x4 with Avastin Q2 weeks/ 2 weeks rest and repeat cycle.'

In [ ]:
import re

def clean_data(data):
  data_clean = {}
  for key in data.keys():
    clean_texts = []
    clean_tag = re.compile('<.*?>') 
    for info in data[key]:
      title = info.split('\n')[0]
      url = info.split('\n')[1]
      text = info.split('\n')[2]
      clean_title = re.sub(clean_tag, '', title)
      clean_text = re.sub(clean_tag, ' ', text)
      clean_text = re.sub('\t', ' ', clean_text)
      all_text = clean_title + ' ' + clean_text
      all_text = re.sub('  +', ' ', all_text)
      clean_texts.append(all_text)
    data_clean[key] = clean_texts

  return data_clean    

In [ ]:
cl_data = clean_data(data1_m)

In [ ]:
cl_data['1'][0]

'Friedreich Ataxia Friedreich ataxia is an inherited disease that damages your nervous system. The damage affects your spinal cord and the nerves that control muscle movement in your arms and legs. Symptoms usually begin between the ages of 5 and 15. The main symptom is ataxia, which means trouble coordinating movements. Specific symptoms include: Difficulty walking Muscle weakness Speech problems Involuntary eye movements Scoliosis (curving of the spine to one side) Heart palpitations, from the heart disease which can happen along with Friedreich ataxia People with Friedreich ataxia usually need a wheelchair 15 to 20 years after symptoms first appear. In severe cases, people become incapacitated. There is no cure. You can treat symptoms with medicines, braces, surgery, and physical therapy. NIH: National Institute of Neurological Disorders and Stroke '

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from string import punctuation

stop = set(stopwords.words('english'))

import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize(text):
  doc = nlp(text)
  return ' '.join([token.lemma_ for token in doc])

def preprocess(text):
  normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
  normalized_text = ' '.join([word.lower() for word in normalized_text if word and word.lower() not in stop and len(word) > 2])
  lemmatized_text = lemmatize(normalized_text)
  return lemmatized_text

In [ ]:
preprocess(cl_data['1'][0])

'friedreich ataxia friedreich ataxia inherit disease damage nervous system damage affect spinal cord nerve control muscle movement arm leg symptom usually begin age main symptom ataxia mean trouble coordinate movement specific symptom include difficulty walk muscle weakness speech problem involuntary eye movement scoliosis curve spine one side heart palpitation heart disease happen along friedreich ataxia people friedreich ataxia usually need wheelchair year symptom first appear severe case people become incapacitate cure treat symptom medicine brace surgery physical therapy nih national institute neurological disorder stroke'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [ ]:
def reranked_ids(corpus, query):
  vectorizer = TfidfVectorizer()

  X = vectorizer.fit_transform(corpus)  

  query_lemmatized = [preprocess(query)]
  query_vector = vectorizer.transform(query_lemmatized)

  sims = []
  for i in range(len(corpus)):
    cossim = cosine_similarity(query_vector, X[i])
    sims.append([i, cossim])
  
  top10 = sorted(sims, key = lambda x: x[1], reverse=True)[:10]

  return [i[0] for i in top10]

In [ ]:
reranked_ids(corpus, queries_clean['1'])

[1, 15, 17, 11, 0, 12, 16, 8, 18, 2]

In [ ]:
new_reranked_recs = {}
for key in queries_clean.keys():
  query = queries_clean[key]
  corpus = [preprocess(text) for text in cl_data[key]]
  new_ids = reranked_ids(corpus, query)
  new_papers = []
  for idx in new_ids:
    new_papers.append(data1_m[key][idx])
  new_reranked_recs[key] = new_papers

In [ ]:
with open('texts_recs_umls_m3_2_clean_reranked.json', 'w') as f:
  json.dump(new_reranked_recs, f)

# recommendation lists evaluation

In [ ]:
import ml_metrics
from sklearn.metrics import ndcg_score as ndcgs

In [ ]:
def top_ndcg(n, all_preds_bin, all_trues_bin):
  pred_bin = [i[:n] for i in all_preds_bin]
  true_bin = [i[:n] for i in all_trues_bin]

  ndcg_res = []
  for pred, true in zip(pred_bin, true_bin):
    ndcg_res.append(ndcg_score(pred, true))

  return np.mean(ndcg_res)

In [ ]:
def get_metrics(all_preds, all_trues):

  all_data = []

  ns = [1, 5, 10]
  for n in ns:
    scores = acc_scores(all_preds, n, all_trues)
    print(f'Top {n} Accuracy = ', np.mean(scores))
    all_data.append(np.mean(scores))

  queries = list(zip(all_preds, all_trues))
  mrr = mean_rr(queries)
  print('Mean Reciprocal Rank = ', mrr)
  all_data.append(mrr)

  all_preds_bin = []
  all_trues_bin = []

  for i in range(len(all_preds)):
    
    i_bins = [1 for i in range(len(all_preds[i]))]
    while len(i_bins) < 2:
      i_bins.append(0)
    all_preds_bin.append(i_bins)

    j_bins = []
    for pred in all_preds[i]:
      if pred in all_trues[i]:
        j_bins.append(1)
      else:
        j_bins.append(0)
    while len(j_bins) < 2:
      j_bins.append(0)
    
    all_trues_bin.append(j_bins)
  
  ndcg_scores = []
  ndcg_scores_2 = []
  for pred, true in zip(all_preds_bin, all_trues_bin):
    ndcg_scores.append(ndcg_score(pred, true))
    ndcg_scores_2.append(ndcgs([true], [pred]))

  #ndcg_mean = np.mean(ndcg_scores)

  #print(f'Mean NDCG = {ndcg_mean}')
  #all_data.append(ndcg_mean)
  #print(f'Mean NDCG top 3 = {top_ndcg(3, all_preds_bin, all_trues_bin)}')
  #all_data.append(top_ndcg(5, all_preds_bin, all_trues_bin))
  #print(f'Mean NDCG top 5 = {top_ndcg(5, all_preds_bin, all_trues_bin)}\n\n')
  #all_data.append(top_ndcg(10, all_preds_bin, all_trues_bin))

  ndcg_mean_2 = np.mean(ndcg_scores_2)
  all_data.append(ndcg_mean_2)
  print(f'Mean NDCG 2 = {ndcg_mean_2}')

  for n in ns:
    all_data.append(ml_metrics.mapk(all_trues, all_preds, n))
    print(f'Top {n} Precision = ', ml_metrics.mapk(all_trues, all_preds, n))

  #return all_data

Загружаем релевантные статьи, выбранные аннотатором

In [ ]:
# первая разметка (QuickUMLS)
with open('true_recs.json', 'r') as f:
  true1 = json.load(f)

true1_list = [true1[str(i)] for i in range(1,76)]

In [ ]:
# вторая разметка (QuickUMLS + Pyserini)
with open('true_recs_2.json', 'r') as f:
  true2 = json.load(f)

true2_list = [true2[str(i)] for i in range(1,76)]

In [ ]:
# комбинация первых двух
with open('true_recs_all.json', 'r') as f:
  all_trues = json.load(f)

all_true_list = [all_trues[str(i)] for i in range(1,76)]

Загружаем списки рекомендаций

In [ ]:
# первый список (QuickUMLS)
with open('texts_recs_with_urls.json', 'r') as f:
  data1 = json.load(f)

umls_list1 = [data1[str(i)] for i in range(1,76)]

In [ ]:
# второй список (QuickUMLS + Pyserini)
with open('text_recs_reranked.json', 'r') as f:
  data2 = json.load(f)

umls_list2 = [data2[str(i)] for i in range(1,76)]

In [ ]:
get_metrics(umls_list1, true1_list)

Top 1 Accuracy =  0.7066666666666667
Top 5 Accuracy =  0.42400000000000004
Top 10 Accuracy =  0.38266666666666665
Mean Reciprocal Rank =  0.8013333333333333
Mean NDCG 2 = 0.5375386096866971
Top 1 Precision =  0.7466666666666667
Top 5 Precision =  0.5493703703703704
Top 10 Precision =  0.706828705803309


In [ ]:
get_metrics(umls_list2, true2_list)

Top 1 Accuracy =  0.5866666666666667
Top 5 Accuracy =  0.5013333333333333
Top 10 Accuracy =  0.4013333333333334
Mean Reciprocal Rank =  0.7271428571428572
Mean NDCG 2 = 0.7010713205978142
Top 1 Precision =  0.6133333333333333
Top 5 Precision =  0.5698703703703704
Top 10 Precision =  0.6824226001511715


In [ ]:
get_metrics(umls_list1, all_true_list)

Top 1 Accuracy =  0.7333333333333333
Top 5 Accuracy =  0.4373333333333334
Top 10 Accuracy =  0.39866666666666667
Mean Reciprocal Rank =  0.8224444444444444
Mean NDCG 2 = 0.625451078158096
Top 1 Precision =  0.7466666666666667
Top 5 Precision =  0.4649407407407408
Top 10 Precision =  0.543437303686907


In [ ]:
get_metrics(umls_list2, all_true_list)

Top 1 Accuracy =  0.6266666666666667
Top 5 Accuracy =  0.5333333333333333
Top 10 Accuracy =  0.43600000000000005
Mean Reciprocal Rank =  0.7545714285714287
Mean NDCG 2 = 0.7323411843809389
Top 1 Precision =  0.64
Top 5 Precision =  0.5405037037037036
Top 10 Precision =  0.5939717330141934


Оценка смешанной системы с уже переранжированными статьями (QuickUMLS + BERT + TF-IDF)

In [ ]:
with open('true_recs_3.json', 'r') as f:
  true3 = json.load(f)

true3_list = [true3[str(i)] for i in range(1,76)]

In [ ]:
with open('texts_recs_umls_m3_2_clean_reranked.json', 'r') as f:
  data_rer = json.load(f)

rer_list = [data_rer[str(i)] for i in range(1,76)]

In [ ]:
get_metrics(rer_list, true3_list)

Top 1 Accuracy =  0.8133333333333334
Top 5 Accuracy =  0.6426666666666666
Top 10 Accuracy =  0.48800000000000004
Mean Reciprocal Rank =  0.8888888888888887
Mean NDCG 2 = 0.7591453183415241
Top 1 Precision =  0.8266666666666667
Top 5 Precision =  0.6495518518518517
Top 10 Precision =  0.7799652053413957


# additional research

Немного изменим вывод функции и посмотрим на крайние значения

In [ ]:
def get_metrics(all_preds, all_trues):

  all_data = []
  return_data = []
  ns = [1, 5, 10]
  for n in ns:
    scores = acc_scores(all_preds, n, all_trues)
    return_data.append(scores)
    print(f'Top {n} Accuracy = ', np.mean(scores))
    all_data.append(np.mean(scores))

  queries = list(zip(all_preds, all_trues))
  mrr = mean_rr(queries)
  print('Mean Reciprocal Rank = ', mrr)
  all_data.append(mrr)
  return_data.append(mrr)

  all_preds_bin = []
  all_trues_bin = []

  for i in range(len(all_preds)):
    
    i_bins = [1 for i in range(len(all_preds[i]))]
    while len(i_bins) < 2:
      i_bins.append(0)
    all_preds_bin.append(i_bins)

    j_bins = []
    for pred in all_preds[i]:
      if pred in all_trues[i]:
        j_bins.append(1)
      else:
        j_bins.append(0)
    while len(j_bins) < 2:
      j_bins.append(0)
    
    all_trues_bin.append(j_bins)
  
  #ndcg_scores = []
  ndcg_scores_2 = []
  for pred, true in zip(all_preds_bin, all_trues_bin):
    #ndcg_scores.append(ndcg_score(pred, true))
    ndcg_scores_2.append(ndcgs([true], [pred]))
  
  return_data.append(ndcg_scores_2)

  ndcg_mean_2 = np.mean(ndcg_scores_2)
  all_data.append(ndcg_mean_2)
  print(f'Mean NDCG 2 = {ndcg_mean_2}')

  for n in ns:
    all_data.append(ml_metrics.mapk(all_trues, all_preds, n))
    print(f'Top {n} Precision = ', ml_metrics.mapk(all_trues, all_preds, n))

  #return all_data
  return return_data

In [ ]:
retdata = get_metrics(umls_list2, true2_list)

Top 1 Accuracy =  0.5866666666666667
Top 5 Accuracy =  0.5013333333333333
Top 10 Accuracy =  0.4013333333333334
Mean Reciprocal Rank =  0.7271428571428572
Mean NDCG 2 = 0.7010713205978142
Top 1 Precision =  0.6133333333333333
Top 5 Precision =  0.5698703703703704
Top 10 Precision =  0.6824226001511715


In [ ]:
#model 1
print(min(retdata[0]), max(retdata[0]))
print(min(retdata[1]), max(retdata[1]))
print(min(retdata[2]), max(retdata[2]))
print(min(retdata[4]), max(retdata[4]))

0.0 1.0
0.0 0.8
0.0 0.9
0.0 0.9208156945822442


In [ ]:
#model 2
print(min(retdata[0]), max(retdata[0]))
print(min(retdata[1]), max(retdata[1]))
print(min(retdata[2]), max(retdata[2]))
print(min(retdata[4]), max(retdata[4]))

0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [ ]:
#model 3
print(min(retdata[0]), max(retdata[0]))
print(min(retdata[1]), max(retdata[1]))
print(min(retdata[2]), max(retdata[2]))
print(min(retdata[4]), max(retdata[4]))

0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [ ]:
with open('umls_ngrams.json', 'r') as f:
  umls_ners = json.load(f)

In [ ]:
with open('umls_m3_2_terms.json', 'r') as f:
  mixed_terms = json.load(f)

In [ ]:
for i in true1.keys():
  if true1[i] == []:
    print(f'T1 = {i} {len(data1[i])} {umls_ners[i]}')
    print(mixed_terms[i])
    print(len(true3[i]))    
  if true2[i] == []:
    print(f'T2 = {i} {len(data2[i])} {umls_ners[i]}')
    print(len(true3[i]))
    print(mixed_terms[i])
  if true3[i] == []:
    print(f'T3 = {i} {len(data_rer[i])} {mixed_terms[i]}')
    print(umls_ners[i])

T2 = 10 6 ['systemic mastocytosis', 'flushing']
3
['systemic mastocytosis', 'flushing', '125 mg iv steroids', 'prenisone taper', 'her home histamine receptor blockers', 'calcium', 'tacardia', 'vitamin d', 'iv steroids', 'adverse reaction', 'another flare', 'steroid taper', 'flare', 'serum tryptase', 'benadryl', 'her home meds', 'iv benadryl', 'gi prophylaxis', 'iv diphenydramine', 'ss bactrim', 'aspirin challenge', 'po steroids']
T1 = 22 24 ['malaise', 'abscess', 'poor appetite', 'leukocytosis', 'abdominal discomfort', 'gangrene', 'tenderness']
['malaise', 'abscess', 'poor appetite', 'leukocytosis', 'abdominal discomfort', 'gangrene', 'tenderness', 'palon of the right lower quadrant', 'pain on the right lab work', 'guarding', 'absces', 'general anesthesia', 'computed tomography of the abdomen with contrast', 'palon of the left lower quadrant', 'distended appendix', 'laparoscopic appendectomy', 'perfor', 'diffuse abdominal discomfort', 'pain', 'focal tenderness', 'thickened appendiceal 

In [ ]:
with open('descriptions.json', 'r') as f:
  texts = json.load(f)

In [ ]:
texts['67']

'\nA Pap smear in a 54-year-old woman revealed abnormal cervical squamous intraepithelial/glandular lesion. She tested positive for HPV 16. She is sexually active with her husband and has 4 children. She is menopausal and uses no contraception. She smokes and drinks alcohol frequently. She is otherwise healthy. She was offered conization.\n'